# Extract data results from generated txts and put into csv for patriotfront

In [ ]:
import pandas as pd
import datetime
from pathlib import Path
import copy

In [ ]:
ROOT_DATA_PATH = "/media/beanbagthomas/AED47533D474FEC1/Large_Storage/data/patriotfrontleak/2021/survai_output/"
CSV_OUT_PATH = "patriotfront_var_results.csv"

output_detections = ['person_on_ground', 'striking', 'spray', 'crowd']

columns = {'file name': str, 'striking count': 0, 'striking timestamps': [], 
    'spray count': 0, 'spray timestamps': [],'crowd count': 0, 'crowd timestamps': [],
    'person_on_ground count': 0, 'person_on_ground timestamps': []}

#max_time_between_ts = 5.0


In [ ]:
files = Path(ROOT_DATA_PATH).glob('*.txt')

df = pd.DataFrame.from_dict(columns)

for file in files:
    det_dictionary = copy.deepcopy(columns)
    det_dictionary['file name'] = str(file).split('/')[-1].split('_out.mp4_detections.txt')[0] + '.'

    with open(file) as f:
        all_txt = f.readlines()

    for det in all_txt:
        det_name = det.split(" ")[0].strip(":")
        det_seconds = int(float(det.split(" ")[-1]))
        if det_seconds <= 5:
            det_timestamp='0:00:00'
        else:
            det_timestamp = str(datetime.timedelta(seconds=det_seconds - 5))
        

        if det_name in output_detections:

            det_dictionary[det_name + ' count'] += 1
            det_dictionary[det_name + ' timestamps'].append(det_timestamp)    
        else:
            continue
    df = df.append(det_dictionary, ignore_index=True)
    f.close()


In [ ]:
df2 = pd.read_csv("patriot_front_leak.csv")

# change filename of our df to filename in leak
for index in range(0, len(df)):
    for ind in range(0, len(df2)):
        if df['file name'][index] in df2['file name'][ind]:
            df['file name'][index] = df2['file name'][ind]
            break

df3 = pd.merge(df, df2, on='file name', how='right')

# reorder
df3 = df3[['index', 'file name', 'directory path', 'link', 'striking count', 'striking timestamps',
 'spray count', 'spray timestamps', 'crowd count', 'crowd timestamps', 
 'person_on_ground count', 'person_on_ground timestamps']]

In [ ]:
df3.to_csv(CSV_OUT_PATH)
df3.sort_values('striking count', ascending=False)